In [1]:
import sys
import os
sys.path.append(os.path.expanduser('~/l/ocgis/src'))
sys.path.append(os.path.expanduser('~/l/esmf/src/addon/ESMPy/src'))
import ocgis
assert(ocgis.__release__ == '2.1.0.dev1')

Configure some environment variables to point to the head directory containing climate data files used in the demo as well as the output directory.

In [2]:
import tempfile
ocgis.env.DIR_DATA = os.path.expanduser('~/l/data/ocgis_test_data/CanCM4')
ocgis.env.DIR_OUTPUT = tempfile.mkdtemp()
print ocgis.env.DIR_OUTPUT

/tmp/tmpMTyAmd


Inspect a target file's metadata.

In [3]:
uri = 'tas_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc'
variable = 'tas'
rd = ocgis.RequestDataset(uri=uri, variable=variable)
rd.inspect()

OCGIS Driver Key: netcdf-cf {
dimensions:
    time = ISUNLIMITED ; // 3650 currently
    lat = 64 ;
    lon = 128 ;
    bnds = 2 ;
variables:
    float64 time(time) ;
      time:bounds = "time_bnds" ;
      time:units = "days since 1850-1-1" ;
      time:calendar = "365_day" ;
      time:axis = "T" ;
      time:long_name = "time" ;
      time:standard_name = "time" ;
    float64 time_bnds(time, bnds) ;
    float64 lat(lat) ;
      lat:bounds = "lat_bnds" ;
      lat:units = "degrees_north" ;
      lat:axis = "Y" ;
      lat:long_name = "latitude" ;
      lat:standard_name = "latitude" ;
    float64 lat_bnds(lat, bnds) ;
    float64 lon(lon) ;
      lon:bounds = "lon_bnds" ;
      lon:units = "degrees_east" ;
      lon:axis = "X" ;
      lon:long_name = "longitude" ;
      lon:standard_name = "longitude" ;
    float64 lon_bnds(lon, bnds) ;
    float64 height() ;
      height:units = "m" ;
      height:axis = "Z" ;
      height:positive = "up" ;
      height:long_name = "height" ;
      

The dimension map defines how metadata is interpreted. This can be customized to deal with non-conforming data or special use cases.

In [4]:
rd.dimension_map.pprint()


{'crs': {'variable': 'latitude_longitude'},
 'driver': 'netcdf-cf',
 'level': {'attrs': {},
           'bounds': None,
           'dimension': [],
           'variable': u'height'},
 'realization': {'dimension': [], 'variable': None},
 'time': {'attrs': {'axis': 'T'},
          'bounds': u'time_bnds',
          'dimension': [u'time'],
          'variable': u'time'},
 'x': {'attrs': {},
       'bounds': u'lon_bnds',
       'dimension': [u'lon'],
       'variable': u'lon'},
 'y': {'attrs': {},
       'bounds': u'lat_bnds',
       'dimension': [u'lat'],
       'variable': u'lat'}}


Subset a target file by the boundary of California using an intersects GIS operation (the default), and write the data to an ESRI Shapefile. Select the first time coordinate only.

In [5]:
geom = os.path.expanduser('~/l/shp/state_boundaries/state_boundaries.shp')
ops = ocgis.OcgOperations(dataset=rd, geom=geom, geom_select_uid=[25], output_format='shp', prefix='ca', 
                          snippet=True)
ca_shp = ops.execute()
print(ca_shp)

/tmp/tmpMTyAmd/ca/ca.shp


 Also write the model grid to shapefile.

In [8]:
ops = ocgis.OcgOperations(dataset=rd, output_format='shp', snippet=True, prefix='grid', spatial_wrapping='wrap')
ca_grid = ops.execute()
print(ca_grid)

/tmp/tmpMTyAmd/grid/grid.shp


Spatially average the grid cells clipped to the boundary of California for all the June, July, and August months in the target dataset. Write the output data to CSV.

In [9]:
import webbrowser
rd = ocgis.RequestDataset(uri=uri, variable=variable, time_region={'month': [6, 7, 8]})
ops = ocgis.OcgOperations(dataset=rd, geom=geom, geom_select_uid=[25], spatial_operation='clip',
                          output_format='csv', prefix='ca_spatial_average', aggregate=True)
ret = ops.execute()
print(ret)
webbrowser.open(ret)

/tmp/tmpMTyAmd/ca_spatial_average/ca_spatial_average.csv


True

Perform a difference calulation between two variables using a string function. Inspect the metadata of the output NetCDF file.

In [11]:
rd1 = ocgis.RequestDataset(uri='tasmax_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc',
                           variable='tasmax')
rd2 = ocgis.RequestDataset(uri='tasmin_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc',
                           variable='tasmin')
calc = 'diff=tasmax-tasmin'
ops = ocgis.OcgOperations(dataset=[rd1, rd2], calc=calc, output_format='nc', geom='state_boundaries',
                          select_ugid=[25], prefix='diff')
ret = ops.execute()
ocgis.RequestDataset(ret).inspect()

/home/benkoziol/l/ocgis/src/ocgis/util/logging_ocgis.py:76: OcgWarning: "vector_wrap" set to False for netCDF output
  warn(exc)


OCGIS Driver Key: netcdf-cf {
dimensions:
    time = ISUNLIMITED ; // 3650 currently
    bnds = 2 ;
    lat = 4 ;
    lon = 4 ;
variables:
    float64 time(time) ;
      time:bounds = "time_bnds" ;
      time:units = "days since 1850-1-1" ;
      time:calendar = "365_day" ;
      time:axis = "T" ;
      time:long_name = "time" ;
      time:standard_name = "time" ;
    float64 time_bnds(time, bnds) ;
      time_bnds:units = "days since 1850-1-1" ;
      time_bnds:calendar = "365_day" ;
    float64 lat(lat) ;
      lat:bounds = "lat_bnds" ;
      lat:units = "degrees_north" ;
      lat:axis = "Y" ;
      lat:long_name = "latitude" ;
      lat:standard_name = "latitude" ;
    float64 lat_bnds(lat, bnds) ;
    float64 lon(lon) ;
      lon:bounds = "lon_bnds" ;
      lon:units = "degrees_east" ;
      lon:axis = "X" ;
      lon:long_name = "longitude" ;
      lon:standard_name = "longitude" ;
    float64 lon_bnds(lon, bnds) ;
    float64 height() ;
      height:units = "m" ;
      height:ax

Calculate a sequence of statistics to produce a July time series conforming the target units from Kelvin to Celsius. Perform the calculations on the spatially averaged data for California.

In [13]:
import webbrowser
rd = ocgis.RequestDataset(uri=uri, variable=variable, time_region={'month': [7]}, conform_units_to='celsius', 
                          field_name='calcs')
calc = [{'func': 'mean', 'name': 'mean'},
        {'func': 'std', 'name': 'stdev'},
        {'func': 'min', 'name': 'min'},
        {'func': 'max', 'name': 'max'},
        {'func': 'median', 'name': 'median'},
        {'func': 'freq_perc', 'name': 'fp_95', 'kwds': {'percentile': 95.0}},
        {'func': 'freq_perc', 'name': 'fp_5', 'kwds':{'percentile': 5.0}},]
calc_grouping = ['month','year']
ops = ocgis.OcgOperations(dataset=rd, geom='state_boundaries', geom_select_uid=[25, 26], spatial_operation='clip',
                          output_format= 'csv', prefix='ca_calcs', aggregate=True, calc=calc,
                          calc_grouping=calc_grouping)
ret = ops.execute()
print(ret)
webbrowser.open(ret)

/tmp/tmpMTyAmd/ca_calcs/ca_calcs.csv


True

Perform the same operation returning the data as a "collection". Print the derived variable aliases.

In [19]:
ops.output_format = 'ocgis'
ret = ops.execute()
print(ret)
print(ret[25].groups[rd.field_name].keys())

<SpatialCollection(Containers :: [25, 26])>
[u'lat', u'lat_bnds', u'lon', u'lon_bnds', u'height', 'ocgis_polygon', 'ocgis_spatial_mask', 'GID', 'latitude_longitude', u'time', 'climatology_bounds', 'mean', 'stdev', 'min', 'max', 'median', 'fp_95', 'fp_5']


Fields are sliceable by dimensions (variables have dimensions similar to NetCDF).

In [33]:
mean = ret.get_element(variable_name='mean')
print('mean dimensions = {}\n'.format(mean.dimensions))
print('mean shape = {}\n'.format(mean.shape))
print('field shapes = {}\n'.format(mean.parent.shapes))
sub = mean.parent[{'time': slice(0, 1)}]
print('sliced field shapes = {}'.format(sub.shapes))

mean dimensions = (Dimension(name='time', size=10, size_current=10, dist=False, is_empty=False), Dimension(name='ocgis_geom_union', size=1, size_current=1, dist=False, is_empty=False))

mean shape = (10, 1)

field shapes = OrderedDict([(u'lat', (4,)), (u'lat_bnds', (4, 2)), (u'lon', (4,)), (u'lon_bnds', (4, 2)), (u'height', ()), ('ocgis_polygon', (1,)), ('ocgis_spatial_mask', (4, 4)), ('GID', (1,)), (u'time', (10,)), ('climatology_bounds', (10, 2)), ('mean', (10, 1)), ('stdev', (10, 1)), ('min', (10, 1)), ('max', (10, 1)), ('median', (10, 1)), ('fp_95', (10, 1)), ('fp_5', (10, 1))])

sliced field shapes = OrderedDict([(u'lat', (4,)), (u'lat_bnds', (4, 2)), (u'lon', (4,)), (u'lon_bnds', (4, 2)), (u'height', ()), ('ocgis_polygon', (1,)), ('ocgis_spatial_mask', (4, 4)), ('GID', (1,)), (u'time', (1,)), ('climatology_bounds', (1, 2)), ('mean', (1, 1)), ('stdev', (1, 1)), ('min', (1, 1)), ('max', (1, 1)), ('median', (1, 1)), ('fp_95', (1, 1)), ('fp_5', (1, 1))])


Print some time values from the time variable.

In [37]:
field = mean.parent
time = field.time
print(time.value_numtime)
print(time.value_datetime)

[55311.0 55676.0 56041.0 56406.0 56771.0 57136.0 57501.0 57866.0 58231.0
 58596.0]
[netcdftime._netcdftime.DatetimeProlepticGregorian(2001, 7, 16, 0, 0, 0, 0, -1, 1)
 netcdftime._netcdftime.DatetimeProlepticGregorian(2002, 7, 16, 0, 0, 0, 0, -1, 1)
 netcdftime._netcdftime.DatetimeProlepticGregorian(2003, 7, 16, 0, 0, 0, 0, -1, 1)
 netcdftime._netcdftime.DatetimeProlepticGregorian(2004, 7, 16, 0, 0, 0, 0, -1, 1)
 netcdftime._netcdftime.DatetimeProlepticGregorian(2005, 7, 16, 0, 0, 0, 0, -1, 1)
 netcdftime._netcdftime.DatetimeProlepticGregorian(2006, 7, 16, 0, 0, 0, 0, -1, 1)
 netcdftime._netcdftime.DatetimeProlepticGregorian(2007, 7, 16, 0, 0, 0, 0, -1, 1)
 netcdftime._netcdftime.DatetimeProlepticGregorian(2008, 7, 16, 0, 0, 0, 0, -1, 1)
 netcdftime._netcdftime.DatetimeProlepticGregorian(2009, 7, 16, 0, 0, 0, 0, -1, 1)
 netcdftime._netcdftime.DatetimeProlepticGregorian(2010, 7, 16, 0, 0, 0, 0, -1, 1)]


Print example variable values.

In [39]:
mean.get_masked_value().squeeze()

masked_array(data=[27.117446899414062, 25.43231201171875,
                   25.081804275512695, 24.113428115844727,
                   26.335866928100586, 24.551790237426758,
                   26.142292022705078, 24.463281631469727,
                   26.58957862854004, 26.490549087524414],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value=1e+20,
            dtype=float32)

Geometries are stored as Shapely objects with associated attributes.

In [44]:
print(type(ret.geoms[25]))
print(ret.geoms[25]).bounds
print(ret[25]['STATE_NAME'].get_value())[0]

<class 'shapely.geometry.multipolygon.MultiPolygon'>
(-124.39263831223747, 32.36153180141197, -114.12523030267519, 41.81088782731149)
California


For three variables, calculate monthly averages for the year 2005 for each U.S. state boundary.

In [48]:
rd1 = ocgis.RequestDataset(uri='tas_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc')
rd2 = ocgis.RequestDataset(uri='tasmin_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc')
rd3 = ocgis.RequestDataset(uri='tasmax_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc')
calc = [{'func': 'mean', 'name': 'mean'}]
calc_grouping = ['month']
ops = ocgis.OcgOperations(dataset=[rd1, rd2, rd3], geom='state_boundaries', aggregate=True,
                          output_format='shp', spatial_operation='clip', prefix='temps',
                          calc=calc, calc_grouping=calc_grouping, time_region={'year': [2005]},
                          conform_units_to='fahrenheit')
ret = ops.execute()
print(ret)

/tmp/tmpMTyAmd/temps/temps.shp


Use ESMF regridding with a subset and spatial aggregation, writing the data to shapefile.

In [ ]:
rd_src = ocgis.RequestDataset(uri='tas_day_CanCM4_decadal2010_r2i1p1_20110101-20201231.nc',
                              variable='tas')
rd_dest = ocgis.RequestDataset(uri='nldas_met_update.obs.daily.pr.1991.nc')
regrid_options = {'with_corners': False}
ops = ocgis.OcgOperations(dataset=rd_src, regrid_destination=rd_dest, geom_select_uid=[6, 16], 
                          agg_selection=True, geom='state_boundaries', snippet=True,
                          output_format='shp', prefix='regrid', regrid_options=regrid_options)
print ops.execute()